# 普通稀疏卷积

In [1]:
import spconv.pytorch as spconv
import torch 

In [5]:
features = torch.tensor([[1], [2], [3]])# your features with shape [N, num_channels]
indices = torch.tensor([[0, 0, 0, 0], [0, 1, 1, 0], [0, 3, 3, 0]], dtype=torch.int32)# your indices/coordinates with shape [N, ndim + 1], batch index must be put in indices[:, 0]
# 第一个位置是batch_idx，后四位是features对应的索引值
spatial_shape = torch.tensor([4, 4, 1])# spatial shape of your sparse tensor, spatial_shape[i] is shape of indices[:, 1 + i].
batch_size = 1# batch size of your sparse tensor.
x = spconv.SparseConvTensor(features, indices, spatial_shape, batch_size)
x_dense_NCHW = x.dense() # convert sparse tensor to dense NCHW tensor.
print(x_dense_NCHW.shape)
print(x_dense_NCHW.dtype)

torch.Size([1, 1, 4, 4, 1])
torch.int64


In [3]:
import spconv.pytorch as spconv
from torch import nn
class ExampleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = spconv.SparseSequential(
            spconv.SparseConv3d(1, 1, 3, 1, 1, bias=False), 
        )

    def forward(self, sparse_input):
        x_sp = spconv.SparseConvTensor.from_dense(sparse_input.reshape(-1, 1, 4, 4, 1))
        return self.net(x_sp)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ExampleNet()
model.net[0].weight.data.fill_(1)
model.to(device)

x_dense_NCHW = x_dense_NCHW.to(torch.float32).to(device) # 必须转类型

output = model(x_dense_NCHW)
output.dense()

tensor([[[[[3., 3., 2., 0.],
           [3., 3., 2., 0.],
           [2., 2., 5., 3.],
           [0., 0., 3., 3.]]]]], device='cuda:0', grad_fn=<PermuteBackward0>)

# 二、SubM卷积

In [11]:
import spconv.pytorch as spconv
from torch import nn
class ExampleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = spconv.SparseSequential(
            spconv.SubMConv3d(1, 1, 3, 1, 1, bias=False), 
        )

    def forward(self, sparse_input):
        x_sp = spconv.SparseConvTensor.from_dense(sparse_input.reshape(-1, 1, 4, 4, 1))
        return self.net(x_sp)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ExampleNet()
model.net[0].weight.data.fill_(1)
model.to(device)

x_dense_NCHW = x_dense_NCHW.to(torch.float32).to(device) # 必须转类型

output = model(x_dense_NCHW)
output.dense()

tensor([[[[[3., 0., 0., 0.],
           [0., 3., 0., 0.],
           [0., 0., 0., 0.],
           [0., 0., 0., 3.]]]]], device='cuda:0', grad_fn=<PermuteBackward0>)